In [ ]:
import pandas as pd
import openai
import time

# 1. CSV 읽기
df = pd.read_csv(
    "C:\\Users\\sunny\\OneDrive\\문서\\이수인\\활동\\창종설\\코카사데이터셋.csv"
)

# 2. OpenAI API Key 설정
client = openai.OpenAI(
    api_key="{[Key]}"
)

# 3. non_sarcasm_explanation 열을 문자열로 변환 (NaN → 빈 문자열)
df["non_sarcasm_explanation"] = df["non_sarcasm_explanation"].fillna("").astype(str)


# 4. GPT로 설명 생성 함수
def generate_explanation(context, response):
    prompt = f"""아래는 두 사람의 대화입니다.
이 대화는 비꼬거나 풍자(sarcasm)가 아니라 진지하고 직접적인 의사소통(non-sarcasm)입니다.
왜 이 대화가 non-sarcasm인지 1~2문장으로 한국어로 간단하게 설명해 주세요.
했습니다 체가 아니라 했다 체로 해주세요.

[대화 내용]
Context (상대방 말): "{context}"
Response (대답): "{response}"

설명:"""
    try:
        gpt_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "당신은 한국어로 대화를 분석하는 어시스턴트입니다.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=1,
            max_tokens=150,
        )
        explanation = gpt_response.choices[0].message.content.strip()
        return explanation
    except Exception as e:
        print(f"❗ 오류 발생 at index {e}")
        return ""


# 5. 1000~1199행만 처리
for idx in range(1000, 1200):
    label_value = str(df.loc[idx, "label"]).strip()
    if label_value.lower() == "non-sarcasm":
        context_text = str(df.loc[idx, "context"])
        response_text = str(df.loc[idx, "response"])
        try:
            generated_explanation = generate_explanation(context_text, response_text)
            df.loc[idx, "non_sarcasm_explanation"] = generated_explanation
            print(f"{idx}번째 Non-Sarcasm 행 처리 완료")
            print(f"{idx}번째 생성된 설명: {generated_explanation}")
        except Exception as e:
            print(f"❗ {idx}번째에서 오류 발생: {e}")
            df.loc[idx, "non_sarcasm_explanation"] = "API_ERROR"

# 6. 결과 저장: 1000~1199행만 별도 저장
output_df = df.loc[1000:1199]
output_path = "C:\\Users\\sunny\\OneDrive\\문서\\이수인\\활동\\창종설\\코카사_1000_to_1199_완성본.csv"
output_df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"✅ 1000~1199행 저장 완료! ({output_path})")

print("🎉 전체 처리 완료!")